In [ ]:
# Third-party
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('notebook.mplstyle')
%matplotlib inline

# Astropy
import astropy.coordinates as coord
import astropy.units as u

Support for transforming velocities and proper motions was added to Astropy in v2.0 (released 2017-07-08), so you may need to update your version of Astropy to get the new features. If you use Anaconda (`conda`) for your Python installation, you can just do:

```
conda update astropy
```

If you use `pip`, you can do:

```
pip install --upgrade astropy
```

If you don't use either, come talk to me...

### Links

Here are two relevant pages in the Astropy documentation:

* [Transform from Heliocentric, spherical velocities (proper motion, etc.) to Galactocentric Cartesian velocities](
http://docs.astropy.org/en/latest/generated/examples/coordinates/plot_galactocentric-frame.html#sphx-glr-generated-examples-coordinates-plot-galactocentric-frame-py)
* [Working with velocities in astropy.coordinates](http://docs.astropy.org/en/latest/coordinates/velocities.html)

## Astropy coordinates terminology

* "Frame": This is the reference frame of your data. For example, ICRS (RA, Dec), Galactic (l, b), etc. All of the typical astronomical reference frames are implemented as classes in `astropy.coordinates`. For example, ICRS is just `ICRS`, etc. See the [transform graph](http://docs.astropy.org/en/latest/coordinates/index.html#module-astropy.coordinates) for a visualization of all of the frames and the transformations that are defined. The coordinate component names for each frame change depending on the type of frame. So, for `ICRS`->(ra,dec), for `Galactic`->(l,b). You can transform between these frames using the `.transform_to()` method:

In [ ]:
icrs = coord.ICRS(ra=150 * u.degree, dec=-11*u.degree)

In [ ]:
icrs.transform_to(coord.Galactic)

All of the values can be arrays or singular values:

In [ ]:
icrs = coord.ICRS(ra=np.arange(10) * u.degree, dec=np.arange(10) * u.degree)
icrs

* "Representation": e.g., Cartesian, spherical, cylindrical. This is just a transformation of the data without a change of reference frame. Each frame has a default representation (e.g., for ICRS, Galactic, it's spherical) but that can be changed.

In [ ]:
icrs = coord.ICRS(ra=150 * u.degree, dec=-11 * u.degree)
icrs.representation

In [ ]:
icrs.set_representation_cls(coord.CylindricalRepresentation)

In [ ]:
icrs.representation

In [ ]:
icrs.rho, icrs.phi

* "Differential": Any derivative of the representation. This is how the velocities are currently stored internally, as time derivatives of the representation object. You don't really need to know about the low-level API, but you may see the word "differential" in example code.

Specifying values to the frame classes makes use of the [`astropy.units`](http://docs.astropy.org/en/latest/units/index.html). The primary object in the units subpackage is the [`Quantity`](http://docs.astropy.org/en/latest/units/quantity.html), which are representations of "number" or "array" data with units. They are typically created by multiplying a unit object with an array or number. For example:

In [ ]:
15 * u.km

In [ ]:
dist = np.arange(10) * u.kpc
dist

Arithmetic with these objects preserves the units and carries them along:

In [ ]:
dist / (15*u.Myr)

And the units can be re-represented using the `.to()` method:

In [ ]:
v = dist / (15*u.Myr)
v.to(u.km/u.s)

----

### Passing velocity components to the coordinate frames

With the default (spherical) representations for most frames, the velocity components are specified as `pm_XX_cosYY`, `pm_YY`, and `radial_velocity`. For example, for the ICRS frame (which is Barycentric):

In [ ]:
coord.ICRS(ra=8.67*u.hourangle, dec=53.09*u.deg,
           pm_ra_cosdec=21*u.mas/u.yr, 
           pm_dec=117*u.mas/u.yr)

In [ ]:
coord.ICRS(ra=8.67*u.hourangle, dec=53.09*u.deg,
           pm_ra_cosdec=21*u.mas/u.yr, 
           pm_dec=117*u.mas/u.yr,
           radial_velocity=71*u.km/u.s)

Only some frame transformations are supported when incomplete position or velocity data are passed. For example, with only sky position and proper motions, we can only transform to other frames that include a spherical rotation:

In [ ]:
icrs = coord.ICRS(ra=8.67*u.hourangle, dec=53.09*u.deg,
                  pm_ra_cosdec=21*u.mas/u.yr, 
                  pm_dec=117*u.mas/u.yr)

icrs.transform_to(coord.Galactic)

Without a distance or a radial velocity, transforming to any frame that requires a position or velocity offset will fail. For example, with the above, we can't transform to a `Galactocentric` frame:

In [ ]:
icrs = coord.ICRS(ra=8.67*u.hourangle, dec=53.09*u.deg,
                  pm_ra_cosdec=21*u.mas/u.yr, 
                  pm_dec=117*u.mas/u.yr)
icrs.transform_to(coord.Galactocentric)

But if we specify these other components:

In [ ]:
icrs = coord.ICRS(ra=8.67*u.hourangle, dec=53.09*u.deg,
                  distance=150*u.pc,
                  pm_ra_cosdec=21*u.mas/u.yr, 
                  pm_dec=117*u.mas/u.yr,
                  radial_velocity=21*u.km/u.s)
gc = icrs.transform_to(coord.Galactocentric)
gc

The default Galactocentric frame has the parameters:

In [ ]:
print(gc.galcen_distance, gc.galcen_v_sun, gc.z_sun)

The default representation for the Galactocentric frame is Cartesian, but we can change to, e.g., cylindrical:

In [ ]:
gc.v_x

In [ ]:
gc.set_representation_cls('cylindrical')

In [ ]:
gc.rho, gc.z

In [ ]:
gc.d_rho.to(u.km/u.s), gc.d_z.to(u.km/u.s)

All of this works with arrays as well. As an example, let's create a circular ring of particles at 15 kpc in the Galactocentric frame with the circular velocity of the disk, and transform to observable coordinates:

In [ ]:
circ_velocity = 220 * u.km/u.s
dist = 15 * u.kpc

phi_grid = np.random.uniform(0, 360, 512) * u.degree # grid of azimuths
ring_rep = coord.CylindricalRepresentation(rho=dist, phi=phi_grid, z=0*u.pc)

angular_velocity = -circ_velocity*np.ones(len(phi_grid)) / dist

ring_dif = coord.CylindricalDifferential(
    d_rho=0*u.km/u.s,
    d_phi=angular_velocity.to(u.mas/u.yr, u.dimensionless_angles()),
    d_z=0*u.km/u.s
)

ring_rep = ring_rep.with_differentials(ring_dif)
gc_ring = coord.Galactocentric(ring_rep)

In [ ]:
icrs_ring = gc_ring.transform_to(coord.ICRS)

In [ ]:
fig,axes = plt.subplots(1, 2, figsize=(12,6))

# Positions
axes[0].plot(gc_ring.x, gc_ring.y, marker='.', linestyle='none')
axes[0].text(-8., 0, r'$\odot$', fontsize=20)

axes[0].set_xlim(-30, 30)
axes[0].set_ylim(-30, 30)

axes[0].set_xlabel('$x$ [kpc]')
axes[0].set_ylabel('$y$ [kpc]')

# Velocities
axes[1].plot(gc_ring.v_x, gc_ring.v_y, marker='.', linestyle='none')

axes[1].set_xlim(-250, 250)
axes[1].set_ylim(-250, 250)

axes[1].set_xlabel('$v_x$ [{0}]'.format((u.km/u.s).to_string("latex_inline")))
axes[1].set_ylabel('$v_y$ [{0}]'.format((u.km/u.s).to_string("latex_inline")))

fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(8,6))

ax.plot(icrs_ring.ra.degree, icrs_ring.pm_ra_cosdec.value,
        marker='.', linestyle='none')

ax.set_xlim(360, 0)

ax.set_xlabel(r'$\alpha$ [deg]')
ax.set_ylabel(r'$\mu_\alpha \, \cos \delta$ [{0}]'.format((u.mas/u.yr).to_string('latex_inline')))

### Defining a custom coordinate frame - Stellar stream coordinates

As an example of how to define your own, custom coordinate frame, we'll define a Sagittarius stream coordinate frame. This system is roughly aligned with the stream so that the longitude component increases along the stream and the latitude is perpendicular to the stream. Though, this is only approximate because it is still a Barycentric frame. 
A more expository version of this tutorial is included [in the documentation](http://docs.astropy.org/en/latest/generated/examples/coordinates/plot_sgr-coordinate-frame.html#sphx-glr-generated-examples-coordinates-plot-sgr-coordinate-frame-py).

Most of the below code is either documentation or boilerplate code that you can copy and paste if you want to define your own. The key bits below are that the longitude and latitude components of the frame are typically defined as $\Lambda$ and $B$, so we'll call them `Lambda` and `Beta` in the below.

In [ ]:
from astropy.coordinates import frame_transform_graph
from astropy.coordinates.matrix_utilities import rotation_matrix, matrix_product, matrix_transpose

In [ ]:
class Sagittarius(coord.BaseCoordinateFrame):
    """
    A Heliocentric spherical coordinate system defined by the orbit
    of the Sagittarius dwarf galaxy, as described in
        http://adsabs.harvard.edu/abs/2003ApJ...599.1082M
    and further explained in
        http://www.stsci.edu/~dlaw/Sgr/.

    Parameters
    ----------
    representation : `BaseRepresentation` or None
        A representation object or None to have no data (or use the other keywords)

    Lambda : `Angle`, optional, must be keyword
        The longitude-like angle corresponding to Sagittarius' orbit.
    Beta : `Angle`, optional, must be keyword
        The latitude-like angle corresponding to Sagittarius' orbit.
    distance : `Quantity`, optional, must be keyword
        The Distance for this object along the line-of-sight.

    pm_Lambda_cosBeta : :class:`~astropy.units.Quantity`, optional, must be keyword
        The proper motion along the stream in ``Lambda`` (including the
        ``cos(Beta)`` factor) for this object (``pm_Beta`` must also be given).
    pm_Beta : :class:`~astropy.units.Quantity`, optional, must be keyword
        The proper motion in Declination for this object (``pm_ra_cosdec`` must
        also be given).
    radial_velocity : :class:`~astropy.units.Quantity`, optional, must be keyword
        The radial velocity of this object.

    """
    default_representation = coord.SphericalRepresentation
    default_differential = coord.SphericalCosLatDifferential

    frame_specific_representation_info = {
        coord.SphericalRepresentation: [
            coord.RepresentationMapping('lon', 'Lambda'),
            coord.RepresentationMapping('lat', 'Beta'),
            coord.RepresentationMapping('distance', 'distance')],
        coord.SphericalCosLatDifferential: [
            coord.RepresentationMapping('d_lon_coslat', 'pm_Lambda_cosBeta'),
            coord.RepresentationMapping('d_lat', 'pm_Beta'),
            coord.RepresentationMapping('d_distance', 'radial_velocity')],
        coord.SphericalDifferential: [
            coord.RepresentationMapping('d_lon', 'pm_Lambda'),
            coord.RepresentationMapping('d_lat', 'pm_Beta'),
            coord.RepresentationMapping('d_distance', 'radial_velocity')]
    }

    frame_specific_representation_info[coord.UnitSphericalRepresentation] = \
        frame_specific_representation_info[coord.SphericalRepresentation]
    frame_specific_representation_info[coord.UnitSphericalCosLatDifferential] = \
        frame_specific_representation_info[coord.SphericalCosLatDifferential]
    frame_specific_representation_info[coord.UnitSphericalDifferential] = \
        frame_specific_representation_info[coord.SphericalDifferential]

In [ ]:
# Euler angles for the transformation from Law & Majewski 2010
SGR_PHI = (180 + 3.75) * u.degree 
SGR_THETA = (90 - 13.46) * u.degree
SGR_PSI = (180 + 14.111534) * u.degree

# Generate the rotation matrix using the x-convention (see Goldstein)
D = rotation_matrix(SGR_PHI, "z")
C = rotation_matrix(SGR_THETA, "x")
B = rotation_matrix(SGR_PSI, "z")
A = np.diag([1.,1.,-1.])
SGR_MATRIX = matrix_product(A, B, C, D)

In [ ]:
@frame_transform_graph.transform(coord.StaticMatrixTransform, coord.Galactic, Sagittarius)
def galactic_to_sgr():
    """ Compute the transformation matrix from Galactic spherical to
        heliocentric Sgr coordinates.
    """
    return SGR_MATRIX

@frame_transform_graph.transform(coord.StaticMatrixTransform, Sagittarius, coord.Galactic)
def sgr_to_galactic():
    """ Compute the transformation matrix from heliocentric Sgr coordinates to
        spherical Galactic.
    """
    return matrix_transpose(SGR_MATRIX)

In [ ]:
icrs = coord.ICRS(ra=8.67*u.hourangle, dec=53.09*u.deg,
                  pm_ra_cosdec=21*u.mas/u.yr, 
                  pm_dec=117*u.mas/u.yr)

icrs.transform_to(Sagittarius)